# 打开视频文件

In [1]:
import os
import cv2

# cap = cv2.VideoCapture("6.avi")

# if not cap.isOpened():
#     print("Error opening video stream or file")

# 读取帧到指定文件夹

In [2]:
frame_dir = "../data/extensometer_frames/"
# frame_interval = 1000

# i = 0
# while cap.isOpened():
#     ret, frame = cap.read()

#     if not ret:
#         print("Can't receive frame (stream end?). Exiting...")
#         break

#     if i % frame_interval == 0:
#         cv2.imwrite("../data/extensometer_frames/" + "img_%s.png" % str(i).zfill(4), frame)

#     i = i + 1

# 图像裁剪

In [3]:
import glob


def list_files_glob(dir_path, ext="*.png"):
    glob_pathname = os.path.join(dir_path, ext)
    print(glob_pathname)
    return glob.glob(glob_pathname, recursive=True)


file_lists = list_files_glob(frame_dir)[1:]

crop_images_dir = os.path.join(frame_dir, "crop/")

for _file in file_lists:
    im = cv2.imread(_file)
    print(im.shape)
    im_crop = im[268:328, 120:]
    crop_pathname = os.path.join(crop_images_dir, f"{os.path.basename(_file)}")
    print(crop_pathname)
    cv2.imwrite(crop_pathname, im_crop)

../data/extensometer_frames/*.png


# 检查红色标记之间的距离

In [4]:
crop_file_lists = list_files_glob(crop_images_dir)

# 颜色检测
import numpy as np


def ps_to_py_hsv_color(ps_hsv_color):
    return ps_hsv_color[0] / 2, ps_hsv_color[1] * 255 / 100, ps_hsv_color[2] * 255 / 100


test_image = cv2.imread(crop_file_lists[0])
hsv_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2HSV)

# lower_red = (156, 43, 46)
# upper_red = (180, 255, 255)
hsv_lower_red = (160, 43, 46)
hsv_upper_red = (175, 255, 255)

# 颜色检测（红色）
hsv_mask = cv2.inRange(hsv_image, hsv_lower_red, hsv_upper_red)

# 中值平滑，消除噪声
hsv_mask_median = cv2.medianBlur(hsv_mask, 9)

cv2.imwrite(os.path.join(crop_images_dir, "test_0.png"), hsv_image)
cv2.imwrite(os.path.join(crop_images_dir, "test_1.png"), hsv_mask)
cv2.imwrite(os.path.join(crop_images_dir, "test_2.png"), hsv_mask_median)

# 找到轮廓
contours, hierarchy = cv2.findContours(hsv_mask_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


# 找出面积最大的两个轮廓
sorted_contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)[0:2]

# 画出找到的两个最大轮廓
# temp = np.ones(hsv_mask_median.shape, np.uint8) * 255
# cv2.drawContours(temp, sorted_contours, -1, (0, 255, 0), 3)
# cv2.imshow("contour", temp)

# 找到最大两个轮廓的边界矩形，并算出他们之间的距离
x0, y0, w0, h0 = cv2.boundingRect(sorted_contours[0])  # x,y,w,h
x1, y1, w1, h1 = cv2.boundingRect(sorted_contours[1])

train_len = abs(x0 + w0 / 2 - x1 - w1 / 2)

print(train_len)

../data/extensometer_frames/crop/*.png
580.0
